In [ ]:
from sqlalchemy import Column, Integer, MetaData, create_engine
from sqlalchemy import Table
import sqlalchemy
import pandas as pd


# Create a connection to the SQLite database
engine = create_engine("sqlite:///Data/CA_COVID_data.sqlite")

# Use pandas to read the migration_map_data table into a DataFrame
df_migration_data = pd.read_sql("SELECT * FROM migration_map_data", engine)

# Print the DataFrame
df_migration_data

In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, PrimaryKeyConstraint

DATABASE_URI = "sqlite:///Data/CA_COVID_data.sqlite"
engine = create_engine(DATABASE_URI)
metadata = MetaData()

# Reflect the existing migration_map_data table
migration_map_data = Table('migration_map_data', metadata, autoload_with=engine)

# Create a new table with the same structure, but with the 'index' column as primary key
new_migration_map_data = Table('new_migration_map_data', metadata,
    Column('index', Integer, primary_key=True),
    *(column.copy() for column in migration_map_data.columns if column.name != 'index'),
    extend_existing=True
)

# Create the new table
new_migration_map_data.create(engine)

# Copy data from the old table to the new table
with engine.connect() as conn:
    select_stmt = migration_map_data.select()
    result = conn.execute(select_stmt)
    for row in result:
        insert_stmt = new_migration_map_data.insert().values(row._asdict())
        conn.execute(insert_stmt)

# Drop the old table
migration_map_data.drop(engine)

# Rename the new table to the original name
with engine.connect() as conn:
    conn.execute("ALTER TABLE new_migration_map_data RENAME TO migration_map_data")



In [ ]:
# Define your database URI
DATABASE_URI = "sqlite:///Data/CA_COVID_data.sqlite"
engine = create_engine(DATABASE_URI)
metadata = MetaData()

# Define the table you want to drop (without creating columns and other details)
table_to_drop = Table('new_migration_map_data', metadata)

# Drop the table
table_to_drop.drop(engine, checkfirst=True)


### Printout a SQLite table  as a dataframe

In [14]:
# Create a connection to the SQLite database
engine = create_engine("sqlite:///Data/CA_COVID_data.sqlite")

# Use pandas to read the desired table into a DataFrame
df = pd.read_sql_table('county_migrations', engine)

# Print the DataFrame
print(df)


          pk     county state  fips_state  fips_county   fips  year  stayput  \
0          0  Abbeville    SC          45            1  45001  2017    16703   
1          1  Abbeville    SC          45            1  45001  2018    16829   
2          2  Abbeville    SC          45            1  45001  2019    16862   
3          3  Abbeville    SC          45            1  45001  2020    16987   
4          4  Abbeville    SC          45            1  45001  2021    16338   
...      ...        ...   ...         ...          ...    ...   ...      ...   
15645  15645    Ziebach    SD          46          137  46137  2017      945   
15646  15646    Ziebach    SD          46          137  46137  2018      942   
15647  15647    Ziebach    SD          46          137  46137  2019      938   
15648  15648    Ziebach    SD          46          137  46137  2020      995   
15649  15649    Ziebach    SD          46          137  46137  2021      955   

       inflow  outflow  net_migration  

### Drop a table

In [17]:
# Define your database URI and set up a connection
DATABASE_URI = "sqlite:///Data/CA_COVID_data.sqlite"
engine = create_engine(DATABASE_URI)
metadata = MetaData()

# Reflect the table you want to drop
table_to_drop = Table('backup_map_data', metadata, autoload_with=engine)

# Drop the table
table_to_drop.drop(engine)


### Convert shapefile into geojson

In [1]:
import geopandas as gpd

# Read the shapefile
gdf = gpd.read_file('Data/SHAPEFILE_US_Census')  # Replace with your Shapefile directory

# Write to GeoJSON format
gdf.to_file('Data/us_counties_geometry.geojson', driver='GeoJSON')


#### Examine geojson file to find missing 'properties'

In [11]:
import json
import pandas as pd
import geopandas as gpd


import json

# Step 1: Read the GeoJSON file
with open("EXAMINE.geojson", "r") as file:
    geojson_data = json.load(file)

# Check if the GeoJSON has the 'features' attribute
if "features" not in geojson_data:
    print("The provided GeoJSON does not have a 'features' attribute.")
    exit()

# Step 2 & 3: Loop through the features and identify ones without 'properties'
features_without_properties = [
    feature for feature in geojson_data["features"] if "properties" not in feature
]

# Print the count of features without properties
print(f"Found {len(features_without_properties)} features without 'properties' attribute.")

# Step 4: Save the identified features in a list (This step is already done in Step 3)
# If you wish to examine or further process these features, they are available in the 'features_without_properties' list.

# If you want to save the 'features_without_properties' list to another JSON file, you can do:
with open("Features_Without_Properties.json", "w") as output_file:
    json.dump(features_without_properties, output_file, indent=4)





TypeError: string indices must be integers